In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.SN_Curve import SN_Curve_qats
from utils.create_geo_matrix import create_geo_matrix
from utils.transformations import global_2_compass, compass_2_global
import numpy as np
import sys
import pandas as pd
import os
from utils.get_scf_sector_list import get_scf_sector_list

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [30 / 2.54, 20 / 2.54]


### Pre-define data paths etc.

In [3]:
# Get relevant data_paths for DLC and simulation result files
data_path              = fr'{os.getcwd()}\data'
geometry_file          = data_path +  r'\DA_P53_CD.xlsx'
point_angles  = [float(i) for i in range(0,359,15)] 

# Extract file for all DEM sums per geo, per elevation
DEM_data_path = fr'{os.getcwd()}\output\python_combined_DEM.xlsx'
df_xlsx = pd.read_excel(DEM_data_path)

In [4]:
geo_test_df = pd.read_excel(fr'{os.getcwd()}\unused\DA_P53_CD copy with extra geo.xlsx')
top_boat_landing_geo = geo_test_df.iloc[0]
elevation_boat_landing = top_boat_landing_geo['z']

In [5]:
# Translate the angles to compass angles to align with given SCF factor
global_angles = [float(key) for key in df_xlsx.iloc[0][1:].keys()] # 1: due to element 0 being 'mLat'
compass_angles = global_2_compass(global_angles)
adjusted_compass_angles, scf_sectors = get_scf_sector_list(top_boat_landing_geo, compass_angles)
adjusted_global_angles = compass_2_global(adjusted_compass_angles)

validate_df = pd.DataFrame(data=np.hstack((np.array([adjusted_compass_angles]).reshape(-1,1), 
                                np.array([scf_sectors]).reshape(-1,1), 
                                np.array([adjusted_global_angles]).reshape(-1,1))), 
                                columns = ['Adjusted compass angles', 'SCF','Adjusted global angles'])

# print(validate_df)
# print(40*'-')

### Extract pre computed DEM data - use boat landing geometry - find elevations of nodes above and below boat landing

In [6]:
# import json
# with open(fr'{os.getcwd()}\output\python_combined_DEM.json') as json_file:
#     DEM_data = json.load(json_file)
'''
The goal here is for this script to be independent of the geometries that were used during the initial DEM-sum calculations
'''
elevations = np.array([float(key) for key in (df_xlsx['mLat'].values)])
dist = elevations - elevation_boat_landing

assert (np.any(dist >= 0.)), 'Geo in question is above the highest node - invalid for interpolation'
assert (np.any(dist < 0.)), 'Geo in question is below the lowest node - invalid for interpolation'

In [7]:
idx_above = np.ma.MaskedArray(dist, dist < 0.0).argmin() # mask out values below 0, and find the positive value closest to zero
idx_below = np.ma.MaskedArray(dist, dist >= 0.0).argmax() # mask out values above 0, and find the negative value closest to zero
idx_closest = np.abs(dist).argmin()

elevation_above = elevations[idx_above]
elevation_below = elevations[idx_below]
elevation_closest = elevations[idx_closest]
                             
print(f'The nodes from the DEM calculation closest to boat landing at {elevation_boat_landing:.2f} mLat is \n{elevation_above:.2f} @ mLat above \n{elevation_below:.2f} @ mLat below')
closest_is_above = elevation_closest > elevation_boat_landing
print(f'The closest of elevations is {elevation_closest}, {"above" if closest_is_above else "below"} the boat landing')

The nodes from the DEM calculation closest to boat landing at 9.69 mLat is 
11.20 @ mLat above 
6.50 @ mLat below
The closest of elevations is 11.2, above the boat landing


### Extract DEM from the neighbouring elevations

In [8]:
# Manually find DEM interpolated for 9.69 mLat by using manually chosen elevations
above_DEM = np.array(df_xlsx.iloc[idx_above][1:]) # remove the mLat dimension of the DataFrame - this will be a np.array with DEM for all (24) sectors
below_DEM = np.array(df_xlsx.iloc[idx_below][1:])

# Linear interpol; DEM_i = DEM_below + [dy/dx] * dx = DEM_below + [(delta DEM above/below) / (delta elevation above/below) ] * (elevation geo i - elevation below) or DEM_below + interpol_factor * delta DEM
interpol_factor = (elevation_boat_landing - elevation_below) / (elevation_above - elevation_below) 
DEM_boat_landing = below_DEM + (above_DEM - below_DEM) * interpol_factor

# Visualize result of interpolation
# pd.DataFrame(data=np.hstack((above_DEM.reshape(-1,1), DEM_boat_landing.reshape(-1,1), below_DEM.reshape(-1,1))), columns = ['Above DEM','Interpol DEM','Below DEM'])

## Use the interpolated DEM to calculate the factor to scale moment ranges with accordingly

In [9]:
# Define DEM variables
T_lifetime = 25.0
N_equivalent = 1e7
wohler_exp = 5.0
curve = SN_Curve_qats('D')
DFF = 3.0

In [10]:
# Calculate DEM for above, current and below elevations, for all sectors
DEM_boat_landing_tot = ((T_lifetime / N_equivalent * DEM_boat_landing)**(1 / wohler_exp))
DEM_above_tot = ((T_lifetime / N_equivalent * above_DEM)**(1 / wohler_exp))
DEM_below_tot = ((T_lifetime / N_equivalent * below_DEM)**(1 / wohler_exp))

In [48]:
# Store the relationsships
print(f'DEM relationships')
print(f'From reports: above vs. hotspot = 99.0 / 98.2 = {99.0 / 98.2:.6f} --- too much rounded')
M_scf_M_above = np.array(DEM_boat_landing_tot / DEM_above_tot)
f_scf_vs_above = M_scf_M_above.max()
print(f'Above : {f_scf_vs_above} at ang {global_angles[ M_scf_M_above.argmax() ]} / {compass_angles[M_scf_M_above.argmax()]} degN')
M_scf_M_below = np.array(DEM_boat_landing_tot / DEM_below_tot)
f_scf_vs_below = M_scf_M_below.max()
print(f'Below : {f_scf_vs_below} at ang {global_angles[ M_scf_M_below.argmax() ] } / {compass_angles[M_scf_M_below.argmax()]} degN')

M_factor = f_scf_vs_above if closest_is_above else f_scf_vs_below
print(f'Chosing factor {M_factor} for {"above" if closest_is_above else "below"}')
M_factor_ang_idx = M_scf_M_above.argmax() if closest_is_above else M_scf_M_below.argmax()


DEM relationships
From reports: above vs. hotspot = 99.0 / 98.2 = 1.008147 --- too much rounded
Above : 1.0135489275462832 at ang 0.0 / 180.0 degN
Below : 0.9746573802655233 at ang 90.0 / 90.0 degN
Chosing factor 1.0135489275462832 for above


### Load the markov matrices with moment ranges and counts

In [12]:
# Load the markov matrices
import json
markov_file_path = fr'{os.getcwd()}\output' + fr'\total_markov.json'
with open(markov_file_path, 'r') as f:
    df_markov = pd.DataFrame.from_dict(json.load(f))

cols = np.array([float(v) for v in df_markov.columns.values])
reference_elevation = elevation_above if closest_is_above else elevation_below
markov_column_idx_reference_elevation = (cols == reference_elevation).nonzero()[0][0]

df_markov_reference = df_markov[str(cols[markov_column_idx_reference_elevation])]
markov_reference_point_ranges = np.array(df_markov_reference[str(M_factor_ang_idx)]) * 1.01 # 1 percent scaling due to pre and post ops

print('Some stats about the reference markov matrix:')
print(f'Number of included bins: {markov_reference_point_ranges.shape[0]}')
print(f'Total counts: {markov_reference_point_ranges[:, 1].sum()}')
print(f'Moment ranges: {markov_reference_point_ranges[:, [0]] / 10**6} ')

M_eq_recreated = ( T_lifetime / N_equivalent * ((markov_reference_point_ranges[:,[0]].T)**wohler_exp).dot(markov_reference_point_ranges[:,[1]]) )**(1 / 5.0)

print(f'Recreated DEM for m = 5: {M_eq_recreated / 10**6 :.2f} MNm')

Some stats about the reference markov matrix:
Number of included bins: 76563
Total counts: 761796.030342383
Moment ranges: [[  0.18034802]
 [  0.54104405]
 [  0.90174008]
 ...
 [107.10550772]
 [109.75008816]
 [112.39466859]] 


### Rescale the moment ranges for the nearest hotspot according to the DEM relation, and find corresponding stress ranges

In [34]:
# Scale all moment ranges per sector according to relationships calculated above, or just one single factor determined by DEM max. Test below, above and closest
markov_reference_point_ranges[:, 0] *= M_factor

print(markov_reference_point_ranges[:, 0] / 10**6)

[  0.18526817   0.5558045    0.92634084 ... 110.02749897 112.74422734
 115.46095571]


In [35]:
# Use moment ranges to calculate stress ranges
D = top_boat_landing_geo['od']
t = top_boat_landing_geo['wt'] / 1000.0
I = np.pi / 64.0 * (D**4 - (D - 2*t)**4)

stress_ranges = markov_reference_point_ranges[:, [0]] * D / (2 * I)

In [36]:
# Scale stress ranges according to SCF
scf_factor = np.max(scf_sectors)
stress_ranges *= scf_factor
stress_ranges_MPa = stress_ranges / 10**6
print(stress_ranges_MPa)

[[ 0.06881183]
 [ 0.20643548]
 [ 0.34405913]
 ...
 [40.8661309 ]
 [41.87517117]
 [42.88421144]]


In [47]:
# Use stress ranges in miner summation for damage (see reports)
stress_ranges_and_counts = np.hstack( (stress_ranges_MPa, markov_reference_point_ranges[:,[1]] ))
D = curve.miner_sum(np.array([[95.05, 122.0]])) * DFF
print(f'Total damage = {D * 100.0:.2e} %')

Total damage = 2.15e-02 %


In [28]:
# Translate to stress through the "equivalent validation method"
# Note that this is not working for our hotspots of interest


### Old stats

In [18]:
# Translate to stress through the "equivalent validation method"
# Note that this is not working for our hotspots of interest
D = top_boat_landing_geo['od']
t = top_boat_landing_geo['wt'] / 1000.0
I = np.pi / 64.0 * (D**4 - (D - 2*t)**4)
S_eq = DEM_boat_landing_tot * D / (2 * I)
S_hs = np.multiply(S_eq, scf_sectors) # elementwise scale stress with SCF

print(f'Max DEM = {DEM_boat_landing_tot.max() / 10**6:.1f} MNm found at {adjusted_compass_angles[DEM_boat_landing_tot.argmax()]:.1f} degN')
print(f'Max S_eq before SCF = {S_eq.max() / 10**6:.1f} MPa found at {adjusted_compass_angles[S_eq.argmax()]:.1f} degN')
print(f'Max S_eq with SCF {S_hs.max() / 10**6:.1f} MPa found at {adjusted_compass_angles[S_hs.argmax()]:.1f} degN')

Max DEM = 97.8 MNm found at 166.7 degN
Max S_eq before SCF = 24.2 MPa found at 166.7 degN
Max S_eq with SCF 36.3 MPa found at 166.7 degN
